In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from IPython.display import Markdown
from getpass import getpass
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# quantization docs: https://huggingface.co/docs/transformers/main/quantization#quantization

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_id, attn_implementation="flash_attention_2", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# , torch_dtype=torch.bfloat16
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
system_prompt = """
You're the world's best personal trainer.
You always provide your clients with all the information needed to become fitter, stronger and healthier through physical training.
You use your science science know and expertise, nutrition advice, and other relevant factors to create workout routines suitable to your clients.
If clients tell you they do not have access to gym equipments, you never fail to recommend exercises that do not require any tool or equipment.
For each exercise you always provide the reps, sets and rest intervals in seconds appropriate for each exercise and the client's fitness level.
You start each workout program with about 5 minutes of warm-up exercises to make the body ready for more strenuous activities and make it easier to exercise.
You end each workout routine with 5 about minutes of cool-down exercises to ease the body, lower the chance of injury, promote blood flow, and reduce stress to the heart and the muscles.
The warm-up and cool-down exercises are always different and they are always appropriate for the muscle group the person wants to train.
You never recommend exercises in the main workout routine in the warm-up or cool-down sections.
Remember, when clients tell you they do not have access to gym equipments, all the exercises you recommend, including the warm-up and cool-down exercises, can be performed without any tool.
You always limit yourself to respond with the list of exercises. You never add any additional comment.

Design the workout based on the following information:
{workout_context}

Output format:
## 🤸 Warp-up:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)
## 🏋️‍♀️ Workout
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)
...
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)
## 🧘 Cool-down:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)
""".strip()

In [ ]:
duration = "45"
muscle_group = "leg"
gender = "male"
level = "beginner"
equipment = "no equipment"

query = f"{duration}-minute {muscle_group} workout for {gender} {level} level {equipment}"

In [ ]:
messages = [
    {
        "role": "user",
        "content": system_prompt,
    },
#     {
#         "role": "user",
#         "content": query
#     }
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt.strip())

In [ ]:
%%time
outputs = pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, top_p=0.95)
# print(outputs[0]["generated_text"].split("<|assistant|>")[-1])

In [ ]:
Markdown(outputs[0]["generated_text"].split("[/INST]")[-1])

### Add Knowledge Base

In [4]:
index_name = "workouts"

pinecone.init(
    api_key=getpass("PINECONE_API_KEY"),
    environment=getpass("PINECONE_ENVIRONMENT")
)

active_indexes = pinecone.list_indexes()
print(active_indexes)

index = pinecone.GRPCIndex(index_name)
print(index.describe_index_stats())

PINECONE_API_KEY ········
PINECONE_ENVIRONMENT ········


['workouts']
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39}},
 'total_vector_count': 39}


In [5]:
embedding_model_name = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(
    model=embedding_model_name,
    openai_api_key=getpass("OPENAI_API_KEY")
)

OPENAI_API_KEY ········


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [6]:
text_field = "text"
index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embedding_model, text_field)

In [ ]:
# query = "shoulder workout"
# vectorstore.similarity_search(query, k=3)

In [10]:
query = "45 minutes legs workout that does not require any equpment"
num_samples = 3

# retrieve most similar workouts to the input query
similar_workouts = vectorstore.similarity_search(query, k=num_samples)

# (optional) random sample a subset of workouts to promote diversity
similar_workouts = random.sample(similar_workouts, num_samples)

# join together the retrieved workouts in a single string
similar_workouts = "\n\n".join([d.page_content for d in similar_workouts])

# print(system_prompt.format(workout_context=similar_workouts))

# build input messages to feed to the LLM
messages = [
    {
        "role": "user",
        "content": system_prompt.format(workout_context=similar_workouts)
    },
#     {
#         "role": "user",
#         "content": query
#     }
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# print(prompt.strip())

In [11]:
%%time

outputs = pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, top_p=0.95)

CPU times: user 19.8 s, sys: 63.9 ms, total: 19.9 s
Wall time: 19.9 s


In [12]:
Markdown(outputs[0]["generated_text"].split("[/INST]")[-1])

 ## 🤸 Warm-up:
- Fast-Paced Walk (3 minutes)
- Hack Squat (1 minute)
- Single-Leg Press (1 minute)
- Leg Extension (1 minute)
- Bodyweight Walking Lunge (1 minute)
- Side Plank (30 seconds, each side)
- Plyometric Push Up (1 minute)
- Rack Lat Stretch (30 seconds)

## 🏋️‍♀️ Workout
- Hack Squat (3 sets, 10-15 reps, 90 seconds rest)
- Single-Leg Press (3 sets, 15 reps per leg, 60 seconds rest)
- Leg Extension (3 sets, 15 reps, 60 seconds rest)
- Box Squat (3 sets, 10-15 reps, 90 seconds rest)
- Sled Pull (3 sets, 15 reps per leg, 60 seconds rest)
- Dumbbell Leg Extension (3 sets, 15 reps, 60 seconds rest)

## 🧘 Cool-down:
- Fast-Paced Walk (3 minutes)
- Bodyweight Standing Calf Raise (1 minute)
- IT Band Foam Roller (1 minute, each side)
- Bodyweight Single-Leg Deadlift (1 minute, each side)
- Prisoner Squat (30 seconds, hold at the bottom)
- Plank (30 seconds)
- Band Pull-Apart (1 minute)
- Seated Zottman Curl (1 minute)
- 90/90 Hip Crossover (1 minute, each side)
- Sit Up (30 seconds)